Welcome to the Bancor Governance Forum. In this forum, the BancorDAO (Decentralized Autonomous Organization) proposes and discusses token Whitelistings, Trading liquidity limits, Fee changes, Bancor Improvement Proposals, and others.

Please head on to the [Information and Templates](https://gov.bancor.network/c/information-and-templates/16) category for more information on the DAO process.

Voting happens on the [BancorDAO Snapshot Page](https://vote.bancor.network/).

[Proposal: Limit on-curve liquidity to max(520 x 7 day fees, 100k BNT)](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876)
=============================================================================================================================================================================

[LEVEL 1 Under Review](https://gov.bancor.network/c/level-1-under-review/14)


![](https://dub2.discourse-cdn.com/standard20/user_avatar/gov.bancor.network/thedavidmeister/90/129_2.png)

[thedavidmeister](https://gov.bancor.network/u/thedavidmeister)

TLDR:

-   We pay IL on 100% of TKN available for trading
-   We get diminishing returns on new fees per unit of trading liquidity
-   If IL > fees then the protocol dies (and is already in crisis due to existing deficits)
-   B3 has the unique ability to set a cap on trading liquidity
-   Taking capital off-curve has a dual benefit of limiting IL exposure AND making it available to other revenue opportunities (native staking etc.)
-   Currently the protocol deficit increases when ETH price increases by ~1.5x annually relative to BNT, making some assumptions, we could lift this to the protocol being profitable up to a ~4x ETH annual moon relative to BNT by limiting the on-curve trading liquidity to 520x 7 day trading fees + implementing some modest staking model (or similar)
-   We should set a simple cap based on the data we have and commission a more sophisticated model in the near future

Proposal:

When we look at the IL curve:

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/optimized/2X/e/e0d10c4f716b77cc23de68cb160db8c6e9dc28e1_2_690x387.png)


We see that the losses due to IL are a % of all the capital deployed on the trading curve.

If TKN moons 2x relative to BNT then the IL is ~10%, a 4x moon has ~20% IL, etc.

Meanwhile there are only so many fees due to trade volume across all of defi and almost all trades through bancor currently are based on arbitrage and aggregators only. That is to say, there is negligible trading being done directly on bancor due to retail etc. as that market is largely monopolized by aggregators and a few platforms that take the lion's share of human traders such as uniswap.

In considering this proposal, please consider mainly how arb bots and aggregators will respond, not human traders, as the former is 10x+ the latter in terms of protocol fee generation.

Import Data

In [0]:
import pandas as pd

filepath = 'https://bancorml.s3.us-east-2.amazonaws.com/preprocessed_tradingliquidity_vs_deficit-3.csv'
df = pd.read_csv(filepath)

Calculate Rolling Average Fee Revenue

In [0]:
# For 7 day rolling averages
df['trade_revenue_real_rolling_7day_avg'] =  df.groupby(['date', 'targetSymbol'])['trade_revenue_real'].rolling(7, min_periods=1).mean().reset_index(drop=True)
df['trade_revenue_usd_rolling_7day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_usd'].rolling(7, min_periods=1).mean().reset_index(drop=True)

# For 30 day rolling averages
df['trade_revenue_real_rolling_30day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_real'].rolling(30, min_periods=1).mean().reset_index(drop=True)
df['trade_revenue_usd_rolling_30day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_usd'].rolling(30, min_periods=1).mean().reset_index(drop=True)
df

Out[29]:

,date,targetSymbol,trade_count,sourceAmount_real,sourceAmount_real_usd,targetAmount_real,targetAmount_real_usd,trade_revenue_real,trade_revenue_usd,bntprice,surplus_tkn,surplus_bnt,surplus_usd,surplus_perc,tknTradingLiquidity_real,bntTradingLiquidity_real,spotRate,price,liquidity_real,stakedBalance_real,poolTokenSupply_real,trade_revenue_real_rolling_7day_avg,trade_revenue_usd_rolling_7day_avg,trade_revenue_real_rolling_30day_avg,trade_revenue_usd_rolling_30day_avg
0,2022-04-20,dai,3,0.003000,9.176236,9.205342,9.140794,0.018448,0.018318,2.296663,-9.221023e+00,-4.026950e+00,-9.248547e+00,-0.000054,9.161156e+04,4.000805e+04,0.436714,2.2900,1.718604e+05,1.718697e+05,1.718697e+05,0.018448,0.018318,0.018448,0.018318
1,2022-04-22,dai,2,0.202159,601.632956,618.054738,601.526021,1.238587,1.205463,2.231059,-6.283286e+02,-2.744009e+02,-6.122045e+02,-0.001828,3.202751e+05,1.404095e+05,0.438403,2.2200,3.437408e+05,3.437500e+05,3.437500e+05,1.238587,1.205463,1.238587,1.205463
2,2022-04-25,eth,1,1628.103621,3614.390039,1.217790,3649.399144,0.002440,7.313425,2.190786,-1.550104e-01,-2.076344e+02,-4.548826e+02,-0.001337,1.039690e+02,1.403459e+05,1349.882510,2.2200,1.169769e+02,1.159120e+02,1.159120e+02,0.002440,7.313425,0.002440,7.313425
3,2022-04-26,dai,3,7525.703648,15846.229453,16368.808603,15957.237984,32.803224,31.978433,2.214641,-2.101020e+04,-9.493450e+03,-2.102459e+04,-0.030559,6.197640e+05,2.918862e+05,0.470963,2.0700,6.828865e+05,6.875000e+05,6.874721e+05,32.803224,31.978433,32.803224,31.978433
4,2022-04-26,eth,1,2403.614895,5071.627429,1.801257,5103.502330,0.003610,10.227460,2.214641,1.366771e+00,1.827579e+03,4.047433e+03,0.005718,2.187025e+02,2.888900e+05,1320.926933,2.0700,2.388591e+02,2.390141e+02,2.390098e+02,0.003610,10.227460,0.003610,10.227460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,2022-07-13,ren,3,1768.630143,1315.906537,11368.242105,1331.755493,22.782048,2.668849,0.439336,-2.925943e+04,-7.788998e+03,-3.421986e+03,-0.074199,3.650763e+05,9.888587e+04,0.270864,0.4518,3.654831e+05,3.943198e+05,3.938701e+05,22.782048,2.668849,22.782048,2.668849
573,2022-07-13,usdc,11,12820.582315,39171.489157,40310.194464,40287.308795,407.173676,406.942508,0.439336,-1.319153e+06,-2.915052e+06,-1.280687e+06,-0.538241,9.223219e+05,2.044195e+06,2.216357,0.4518,1.112240e+06,2.450837e+06,2.447838e+06,407.173676,406.942508,407.173676,406.942508
574,2022-07-13,vbnt,13,17417.396004,17347.157459,49549.095840,17188.881593,754.554759,261.759618,0.439336,-5.969363e+04,-4.785695e+04,-2.102528e+04,-0.015282,1.550398e+06,1.250088e+06,0.806302,0.4518,3.851069e+06,3.906057e+06,3.889583e+06,754.554759,261.759618,754.554759,261.759618
575,2022-07-13,wbtc,8,77132.736065,40043.358891,2.075963,39984.699361,0.004160,80.128950,0.439336,-1.634478e+02,-7.133886e+06,-3.134172e+06,-0.483749,1.746781e+02,7.624048e+06,43646.270867,0.4518,1.746781e+02,3.378767e+02,3.378000e+02,0.004160,80.128950,0.004160,80.128950


[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#current-situation-1)Current situation
-----------------------------------------------------------------------------------------------------------------------------------------------

Let's look at how that translates to the current state of play for ETH.

There is 8807 ETH on the trading curve right now:

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/original/2X/7/7fc569f4bf2f391578b84463ea82683fd815b20d.png)

Get the latest date in the dataset:

In [0]:
latest_date = df.sort_values('date', ascending=False)['date'].values[0]
latest_date

Out[30]: '2022-07-13'

Based on the above, set `oncurve_eth` and `token_symbol` constants:

In [0]:
token_symbol = 'eth'

In [0]:
oncurve_eth = int(round(df[(df['date']==latest_date) & (df['targetSymbol']==token_symbol)]['tknTradingLiquidity_real'].values[0]))
oncurve_eth

Out[32]: 9724

30 day rolling average ETH fees are:

In [0]:
eth_rolling_30day_real_avg = df[(df['date']==latest_date) & (df['targetSymbol']==token_symbol)]['trade_revenue_real_rolling_30day_avg'].values[0]

eth_rolling_30day_usd_avg = df[(df['date']==latest_date) & (df['targetSymbol']==token_symbol)]['trade_revenue_usd_rolling_30day_avg'].values[0]

print(f'\033[1m 30 day rolling avg: {eth_rolling_30day_real_avg} (real/day) \033[1m \n')

30 day rolling avg: 0.5032507641136772 (real/day)

In [0]:
print(f'\033[1m 30 day rolling avg: {eth_rolling_30day_usd_avg} (usd/day) \033[1m')

30 day rolling avg: 532.0927496151701 (usd/day)

Estimated 7 day ETH fees are:

In [0]:
estimated_7day_eth_fees_real = eth_rolling_30day_real_avg * 7
estimated_7day_eth_fees_usd = eth_rolling_30day_usd_avg * 7

print(f'\033[1m Estimated 7 day ETH fees: {estimated_7day_eth_fees_real} (real) \033[1m \n')

Estimated 7 day ETH fees: 3.5227553487957404 (real)

In [0]:
print(f'\033[1m Estimated 7 day ETH fees: {estimated_7day_eth_fees_usd} (usd) \033[1m')

Estimated 7 day ETH fees: 3724.649247306191 (usd)

Comparing the above to prior naive estimate (limited data):

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/original/2X/3/3b2d3eb390e5564193283285bad0d7a4a333cf53.png)

If we annualize the estimated fees we get:

In [0]:
eth_annualized = round(52 * estimated_7day_eth_fees_real, 2)
eth_annualized_percent_of_oncurve = round((eth_annualized / oncurve_eth) * 100, 2)

print(f'\033[1m ETH Fees annualized: {eth_annualized} (real) \033[1m \n')

ETH Fees annualized: 183.18 (real)

In [0]:
print(f'\033[1m ETH Fees annualized as a percent of ETH on curve: {eth_annualized_percent_of_oncurve} (%) \033[1m')

ETH Fees annualized as a percent of ETH on curve: 1.88 (%)

Setting aside mechanics like leveraged vortexing for the moment, the baseline is that the protocol will break even on ETH if IL on 8800 ETH is 1.29% or less, relative to BNT.

That means that if ETH increases by just 50% relative to BNT over a year, all else equal, the protocol loses ETH and the deficit increases.

[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#proposed-2)Proposed
-----------------------------------------------------------------------------------------------------------------------------

Let's limit the ETH pool size to 520x the 7 day fees, i.e. an APR of 10% all else equal.

This gives us a limited trading liquidity of:

In [0]:
limited_pool_size_520 = int(round(520 * estimated_7day_eth_fees_real, 0))
print(f'\033[1m {limited_pool_size_520} (ETH) \033[1m')

1832 (ETH)

If we assume that the fees generated on the limited trading liquidity are not significantly less than the fees on the current on-curve liquidity then this gives us an APR of 10% on ETH.

### TODO: Below not yet completed.

We spent all of bancor v2.1 with caps on trading pools showing that 10% APR is sustainable for many TKN, but the reality of how many fees we would miss on a 2400 ETH pool vs. an 8800 ETH pool is something that we would have to measure and monitor.

Continuing with the assumption of comparable fees with the limited pool...

This now means that a 2x in ETH vs. BNT costs the protocol 240 ETH instead of 880 ETH. That is to say, over 1 year we can expect the protocol to break even on the 10% APR even if ETH moons 2x relative to BNT.

We also have 6400 ETH freed up to generate fees. Native staking on ETH yields about 3%.

6400 ETH x 0.03 = 192 ETH ~= 200 ETH

Assuming we do the most conservative thing other than doing nothing at all, use the native ETH staking mechanism, we now have a budget of 200 ETH from staking 6400 ETH and 240 ETH from a 10% APR on 2400 ETH to cover the IL on 2400 ETH.

With 440 ETH we can cover up to ~18% IL on 2400 ETH which allows ETH to annually moon ~4x relative to BNT

Clearly a 4x annual moon is a significantly larger safety margin than a 1.5x annual moon.

All we need to do is:

-   Limit pool to 520x the 7 day fees to give a minimum 10% APR
-   Move the off-curve liquidity into a conservative yield generating mechanism offering 3% e.g. native staking

Generally we could and should apply this 520x cap to all TKN on the platform as a rule of thumb and then backfill more sophisticated analytics over time.

For many TKN we might run into a situation where the 7 day fees are too small to support trading of any size, so we can set a minimum, which right now can be somewhat arbitrary like 100k BNT.

[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#voting-3)Voting
-------------------------------------------------------------------------------------------------------------------------

YES: Cap all on-curve TKN trading liquidity to 520x their rolling 7 day fees

-   Frequency and level of automation of the cap management TBD and improved over time
-   Commit to alternative revenue generation for the off-curve capital
-   Commit to monitoring and ongoing analysis that is more sophisticated than what is presented above

NO: Continue to attempt to cover double digit annual % IL with single digit % annual TKN fees